In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
import warnings
warnings.filterwarnings('ignore')

# # Load the cleaned data file
# cwd = os.getcwd()
# print(cwd)
# parent_dir = os.path.dirname(cwd)
# print(parent_dir)
filename = r'C:\Users\jbh\Desktop\NYPD_Complaint_Data_Cleaned.csv'
df = pd.read_csv(filename)


df.shape

C:\Users\jbh\AppData\Local\Temp\ipykernel_18724\2503328965.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(8170556, 30)

In [4]:
# Focus crimes: FELONY ASSAULT, ROBBERY, RAPE
focus_crimes = ['FELONY ASSAULT', 'ROBBERY', 'RAPE']

df_focus = df[df['Offense_Description'].isin(focus_crimes)]


# Keep columns: Complaint_From_Date, Complaint_From_Time, Lattitude, Longtitude, Victim_Sex
df_focus = df_focus[['Complaint_From_Date', 'Complaint_From_Time', 'Latitude', 'Longitude', 'Victim_Sex']]

# Drop rows where Victim_Sex is not "M" or "F" (D=Business/Organization and E = PSNY/People of the State of New York which is not relevant to the case)
df_focus = df_focus[df_focus['Victim_Sex'] != 'D']
df_focus = df_focus[df_focus['Victim_Sex'] != 'E']
# The footnotes doesn't provide any useful information on the Victim_Sex == "L". We think it might be "Lady" or "LQBTQ" but we are not sure. So we will drop these rows. 
# In this subset of data, they only account for approx 800 rows out of 800k rows. So it's not a big deal to drop them.
df_focus = df_focus[df_focus['Victim_Sex'] != 'L']


# convert to datetime
df_focus['Complaint_From_Date'] = pd.to_datetime(df_focus['Complaint_From_Date'])
# create a column "DayOfWeek" and remove the column "Complaint_From_Date"
df_focus['weekday'] = df_focus['Complaint_From_Date'].dt.day_name()
df_focus = df_focus.drop(columns=['Complaint_From_Date'])

# convert Complaint_From_Time to datetime
df_focus['Complaint_From_Time'] = pd.to_datetime(df_focus['Complaint_From_Time'], format='%H:%M:%S').dt.time
# create a column "hour" and remove the column "Complaint_From_Time"
df_focus['hour'] = pd.to_datetime(df_focus['Complaint_From_Time'], format='%H:%M:%S').dt.hour
df_focus = df_focus.drop(columns=['Complaint_From_Time'])

# convert latitude and longitude to float
df_focus['Latitude'] = df_focus['Latitude'].astype(float)
df_focus['Longitude'] = df_focus['Longitude'].astype(float)

# rename to "lat" and "lon"
df_focus = df_focus.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})

# only keep 10k rows for now
# df_focus = df_focus.sample(n=10000)

# save the df to json with comma separated entries encapulated in square brackets
df_focus.to_json('data2.json', orient='records', lines=False)